In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
import numpy as np
import threading
from concurrent.futures import ThreadPoolExecutor

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Final Project/mfp-diaries.tsv', sep='\t', header=None)
data.head()

,0,1,2,3
0,1,2014-09-14,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [4]:
def convert_to_json(row):
    id = row[0]
    date = row[1]
    details = json.loads(row[2])
    summary = json.loads(row[3])
    return pd.DataFrame([[id, date, details, summary]], columns=['Id', 'Date', 'Details', 'Summary'])

In [5]:
def create_threads_for_rows(data):
    threads = []
    results = []

    def thread_function(index, row):
        processed_row = convert_to_json(row)
        results.append(processed_row)

    num_rows = data.shape[0]
    for i in range(num_rows):
        row = data.iloc[i, :]
        thread = threading.Thread(target=thread_function, args=(i, row))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    processed_data = pd.concat(results, ignore_index=True)
    return processed_data

In [6]:
def parse_data(row):
    name_lst = []
    nutrishes_name_lst = []
    nutrishes_value_lst = []

    id = row.iloc[:, 0].values[0]
    date = row.iloc[:, 1].values[0]
    meal_data = row.iloc[:, 2].values[0]
    meal = meal_data[0].get('meal', np.nan)
    dishes = meal_data[0].get('dishes', [])
    detail_len = len(dishes)

    for i in range(detail_len):
        dish = dishes[i]
        nutritions = dish.get('nutritions', [])
        for j in range(len(nutritions)):
            nutrition = nutritions[j]
            nutrishes_name = nutrition.get('name', np.nan)
            nutrishes_value = nutrition.get('value', np.nan)
            nutrishes_name_lst.append(nutrishes_name)
            nutrishes_value_lst.append(nutrishes_value)
            name = dish.get('name', np.nan)
            name_lst.append(name)

    total_data = row.iloc[:, 3].values[0]
    total = total_data.get('total', [{}]*6)
    goal = total_data.get('goal', [{}]*6)

    total_calories = total[0].get('value', np.nan) if len(total) > 0 else np.nan
    total_carbs = total[1].get('value', np.nan) if len(total) > 1 else np.nan
    total_fats = total[2].get('value', np.nan) if len(total) > 2 else np.nan
    total_proteins = total[3].get('value', np.nan) if len(total) > 3 else np.nan
    total_sodiums = total[4].get('value', np.nan) if len(total) > 4 else np.nan
    total_sugars = total[5].get('value', np.nan) if len(total) > 5 else np.nan

    goal_calories = goal[0].get('value', np.nan) if len(goal) > 0 else np.nan
    goal_carbs = goal[1].get('value', np.nan) if len(goal) > 1 else np.nan
    goal_fats = goal[2].get('value', np.nan) if len(goal) > 2 else np.nan
    goal_proteins = goal[3].get('value', np.nan) if len(goal) > 3 else np.nan
    goal_sodiums = goal[4].get('value', np.nan) if len(goal) > 4 else np.nan
    goal_sugars = goal[5].get('value', np.nan) if len(goal) > 5 else np.nan

    sequence = meal_data[0].get('sequence', np.nan)
    len_name = len(nutrishes_name_lst)
    len_value = len(nutrishes_value_lst)
    id_lst = [id] * len_name
    date_lst = [date] * len_name
    meal_lst = [meal] * len_name
    sequence_lst = [sequence] * len_name
    total_calories_lst = [total_calories] * len_name
    total_carbs_lst = [total_carbs] * len_name
    total_fats_lst = [total_fats] * len_name
    total_proteins_lst = [total_proteins] * len_name
    total_sodiums_lst = [total_sodiums] * len_name
    total_sugars_lst = [total_sugars] * len_name

    goal_calories_lst = [goal_calories] * len_name
    goal_carbs_lst = [goal_carbs] * len_name
    goal_fats_lst = [goal_fats] * len_name
    goal_proteins_lst = [goal_proteins] * len_name
    goal_sodiums_lst = [goal_sodiums] * len_name
    goal_sugars_lst = [goal_sugars] * len_name

    return pd.DataFrame({
        'Id': id_lst,
        'Date': date_lst,
        'Meal': meal_lst,
        'Nutrition_Name': nutrishes_name_lst,
        'Value': nutrishes_value_lst,
        'Name': name_lst,
        'Sequence': sequence_lst,
        'Total_Calories': total_calories_lst,
        'Total_Carbs': total_carbs_lst,
        'Total_Fats': total_fats_lst,
        'Total_Protein': total_proteins_lst,
        'Total_Sodium': total_sodiums_lst,
        'Total_Sugar': total_sugars_lst,
        'Goal_Calories': goal_calories_lst,
        'Goal_Carbs': goal_carbs_lst,
        'Goal_Fats': goal_fats_lst,
        'Goal_Protein': goal_proteins_lst,
        'Goal_Sodium': goal_sodiums_lst,
        'Goal_Sugar': goal_sugars_lst
    })


In [7]:
import threading
import queue

def create_threads_parse_rows(data):
    threads = []
    results_queue = queue.Queue()

    def thread_function(index, row):
        processed_row = parse_data(row)
        results_queue.put(processed_row)

    num_rows = data.shape[0]
    for i in range(num_rows):
        row = pd.DataFrame(data.iloc[i, :]).T
        thread = threading.Thread(target=thread_function, args=(i, row))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    results = []
    while not results_queue.empty():
        results.append(results_queue.get())

    final = pd.concat(results, ignore_index=True) if results else pd.DataFrame()
    return final


In [8]:
import pandas as pd
import gc
import os

# Function to process, parse, sort, and save data in batches
def process_and_save_in_batches(data, batch_size=40000, save_dir='/content/drive/MyDrive/Final Project/'):
    os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists

    num_rows = len(data)
    num_batches = num_rows // batch_size + 1  # Calculate number of batches needed

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_rows)

        partial_data = data.iloc[start_idx:end_idx]

        processed_partial_data = pd.DataFrame(create_threads_for_rows(partial_data))

        parsed_data = create_threads_parse_rows(processed_partial_data)

        parsed_data.sort_values(by=['Id', 'Date'], inplace=True)

        batch_filename = f'batch_{i + 1}.csv'
        batch_path = os.path.join(save_dir, batch_filename)

        parsed_data.to_csv(batch_path, index=False)

        print(f'Saved batch {i + 1} to {batch_path}')

        del processed_partial_data
        del parsed_data
        gc.collect()

    print('All batches processed and saved successfully.')
process_and_save_in_batches(data)

Saved batch 1 to /content/drive/MyDrive/Final Project/batch_1.csv
Saved batch 2 to /content/drive/MyDrive/Final Project/batch_2.csv
Saved batch 3 to /content/drive/MyDrive/Final Project/batch_3.csv
Saved batch 4 to /content/drive/MyDrive/Final Project/batch_4.csv
Saved batch 5 to /content/drive/MyDrive/Final Project/batch_5.csv
Saved batch 6 to /content/drive/MyDrive/Final Project/batch_6.csv
Saved batch 7 to /content/drive/MyDrive/Final Project/batch_7.csv
Saved batch 8 to /content/drive/MyDrive/Final Project/batch_8.csv
Saved batch 9 to /content/drive/MyDrive/Final Project/batch_9.csv
Saved batch 10 to /content/drive/MyDrive/Final Project/batch_10.csv
Saved batch 11 to /content/drive/MyDrive/Final Project/batch_11.csv
Saved batch 12 to /content/drive/MyDrive/Final Project/batch_12.csv
Saved batch 13 to /content/drive/MyDrive/Final Project/batch_13.csv
Saved batch 14 to /content/drive/MyDrive/Final Project/batch_14.csv
Saved batch 15 to /content/drive/MyDrive/Final Project/batch_15.cs

In [1]:
def consolidate_batches_into_csv(save_dir='/content/drive/MyDrive/Final Project/'):

    csv_files = [f for f in os.listdir(save_dir) if f.startswith('batch_') and f.endswith('.csv')]
    consolidated_data = pd.DataFrame()
    for csv_file in csv_files:
        csv_path = os.path.join(save_dir, csv_file)

        batch_data = pd.read_csv(csv_path, low_memory=False)

        consolidated_data = pd.concat([consolidated_data, batch_data], ignore_index=True)


    consolidated_file_path = os.path.join(save_dir, 'consolidated_data.csv')
    consolidated_data.to_csv(consolidated_file_path, index=False)

    print(f'Consolidated data saved to {consolidated_file_path}')

consolidate_batches_into_csv()

Consolidated data saved to /content/drive/MyDrive/Final Project/consolidated_data.csv


In [3]:
Consolidated_data = pd.read_csv('/content/drive/MyDrive/Final Project/consolidated_data.csv')
Consolidated_data.head()

,Id,Date,Meal,Nutrition_Name,Value,Name,Sequence,Total_Calories,Total_Carbs,Total_Fats,Total_Protein,Total_Sodium,Total_Sugar,Goal_Calories,Goal_Carbs,Goal_Fats,Goal_Protein,Goal_Sodium,Goal_Sugar
0,1,2014-09-14,MY food,Calories,412,"my - McDonalds Espresso Pronto® Flat White, 2 ...",1,2924,340.0,114.0,186.0,3658.0,109.0,3173.0,396.0,105.0,160.0,2300.0,119.0
1,1,2014-09-14,MY food,Carbs,29,"my - McDonalds Espresso Pronto® Flat White, 2 ...",1,2924,340.0,114.0,186.0,3658.0,109.0,3173.0,396.0,105.0,160.0,2300.0,119.0
2,1,2014-09-14,MY food,Fat,24,"my - McDonalds Espresso Pronto® Flat White, 2 ...",1,2924,340.0,114.0,186.0,3658.0,109.0,3173.0,396.0,105.0,160.0,2300.0,119.0
3,1,2014-09-14,MY food,Protein,21,"my - McDonalds Espresso Pronto® Flat White, 2 ...",1,2924,340.0,114.0,186.0,3658.0,109.0,3173.0,396.0,105.0,160.0,2300.0,119.0
4,1,2014-09-14,MY food,Sodium,258,"my - McDonalds Espresso Pronto® Flat White, 2 ...",1,2924,340.0,114.0,186.0,3658.0,109.0,3173.0,396.0,105.0,160.0,2300.0,119.0
